# Finding Donors for *CharityML*
## Feature Engineering
### Kebei Jiang 06/04/2019

### Goal  
 * benchmark ft engineering: standard normalization and scaling, no discarding or regrouping  
 * EDA inspired ft engineering: with discarding and regrouping

In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

# Import libraries for visulization
import matplotlib.pyplot as plt
import seaborn as sns

#plt.style.use('ggplot')
%matplotlib inline

sns.set(color_codes=True)

In [2]:
# Load the Census dataset
data = pd.read_csv("census.csv")
ft_num = data.select_dtypes(include=['int64','float64']).columns.values
ft_cat = data.select_dtypes(exclude=['int64','float64']).columns.values

## 1st round EDA decisions

| feature-numerical | 1st round decision        |
|-------------------|---------------------------|
| age               | as-is                     |
| education-num     | as-is                     |
| capital-gain      | divide into zero/non-zero |
| capital-loss      | divide into zero/non-zero |
| hours-per-week    | as-is                     |








| feature-categorical | 1st round decision                                                                                         | reasoning                                                         |
|---------------------|------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------|
| workclass           | {without-pay}, {*-gov, private, self-emp-not-inc}, {self-emp-inc}                                          | '-gov', 'private' and 'self-emp-not-inc' are all paid employees   |
| occupation          | {Exec-managerial, prof-specialty}, {farming, handlers, machine-op, other-service, priv-house-serv}, {rest} | based on income classes ratio                                     |
| marital-status      | {Married-AF-spouse, Married-civ-spouse}, {rest}                                                            |                                                                   |
| relationship        | {Husband, Wife}, {rest}                                                                                    | correlation with marital-status, should include both or just one? |
| race                | {Asian-Pac-Islander, white}, {rest}                                                                        | based on income classes ratio                                     |
| sex                 | as-is                                                                                                      |                                                                   |
| native-country      | drop                                                                                                       | just assume everyone is from US                                   |
| education-level      | drop                                                                                                       | duplicated                                   |










 [ref 1](http://scg.sdsu.edu/dataset-adult_r/)
 * Capital-gain/loss into low/high groups
 * combine government works; self-employed...  
 * 'occupation' to 'blue collar' and 'white collar'  
 * 'native-cournty' into continents  
 * scaling/normalizing features  
 * put all feature engineering into a function

[ref 2](https://faculty.biu.ac.il/~yahavi1/Projects/CP2010T1_rep.pdf)  
 * visualize DT  
 * average hours-per-week w.r.t. Gender (married or not)  
 * check predictive error in different classes  

[ref3](http://rstudio-pubs-static.s3.amazonaws.com/265200_a8d21a65d3d34b979c5aafb0de10c221.html)  
Capital gain:

We mark all values of “capital_gain” which are less than the first quartile of the nonzero capital gain (which is equal to 3464) as “Low”; all values that are between the first and third quartile (between 3464 and 14080) - as “Medium”; and all values greater than or equal to the third quartile are marked as “High”.


Asia_East <- c(" Cambodia", " China", " Hong", " Laos", " Thailand",
               " Japan", " Taiwan", " Vietnam")

Asia_Central <- c(" India", " Iran")

Central_America <- c(" Cuba", " Guatemala", " Jamaica", " Nicaragua", 
                     " Puerto-Rico",  " Dominican-Republic", " El-Salvador", 
                     " Haiti", " Honduras", " Mexico", " Trinadad&Tobago")

South_America <- c(" Ecuador", " Peru", " Columbia")


Europe_West <- c(" England", " Germany", " Holand-Netherlands", " Ireland", 
                 " France", " Greece", " Italy", " Portugal", " Scotland")

Europe_East <- c(" Poland", " Yugoslavia", " Hungary")

----

## benchmark feature engineering

In [3]:
from sklearn.preprocessing import MinMaxScaler

def ft_num_engineer(data, ft_num):

    # logrithmic transform on 'capital-gain' and 'capital-loss'
    data['capital-gain']=np.log(data['capital-gain'] + 1)
    data['capital-loss']=np.log(data['capital-loss'] + 1)
    
    # scaling the features
    # scaling works on multiple featurs simultaneously
    scaler = MinMaxScaler()
    data[ft_num] = scaler.fit_transform(data[ft_num])
    
    return data

In [4]:
def ft_cat_eda(data, workclass, occupation, marital, relationship, race):

    # workclass
    workclass_dict = {' Without-pay':'without-pay',' State-gov':'employee', ' Federal-gov':'employee', ' Local-gov':'employee', \
                      ' Private':'employee', ' Self-emp-not-inc':'employee', ' Self-emp-inc':'owner'}
    # occupation
    occupation_income = pd.Series([0, 1, 1, 2, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1]).map({0:'low', 1:'mid', 2:'high'})
    occupation_dict = dict(zip(sorted(data['occupation'].unique()), occupation_income))
    # marital-status
    marital_group = pd.Series([1 if x in [' Married-civ-spouse', ' Married-AF-spouse'] else 0 for x in data['marital-status'].unique()]).map({0:'single', 1:'couple'})
    marital_dict = dict(zip(data['marital-status'].unique(), marital_group))
    # relationship
    relationship_group = pd.Series([1 if x in [' Husband', ' Wife'] else 0 for x in data['relationship'].unique()]).map({0:'single', 1:'couple'})
    relationship_dict = dict(zip(data['relationship'].unique(), relationship_group))
    # race
    race_dict = {' White': 'high', ' Asian-Pac-Islander': 'high', ' Black':'low', ' Amer-Indian-Eskimo':'low', ' Other':'low'}
    
    # replacement
    filter = np.array([workclass, occupation, marital, relationship, race])
    fts = np.array(['workclass', 'occupation', 'marital-status', 'relationship', 'race'])
    dicts = np.array([workclass_dict, occupation_dict, marital_dict, relationship_dict, race_dict])
    
    replace_dict = dict(zip(fts[filter], dicts[filter]))

    data = data.replace(replace_dict)
    
    return data

def ft_engineer(data, capital=False, workclass=False, occupation=False, marital=False, \
                    relationship=False, race=False, drop_native_country=False):

    # numerical engineering
    ft_num = data.select_dtypes(include=['int64','float64']).columns.values    
    data = ft_num_engineer(data, ft_num)
      
    # should we binarize 'capital'
    if capital:
        data['capital-gain']= data['capital-gain'].apply(lambda x: 'no' if x==0 else 'yes')
        data['capital-loss']= data['capital-loss'].apply(lambda x: 'no' if x==0 else 'yes')
    
    # EDA suggested update
    data = ft_cat_eda(data, workclass, occupation, marital, relationship, race)
    
    # should we drop 'native-country'
    if drop_native_country:
        data.drop('native-country', axis=1, inplace=True)

    # target and get_dummies
    target = np.array(data['income'] != '<=50K').astype(int)
    
    #data.drop(['education_level', 'income'], axis=1, inplace=True)
    data.drop(['income'], axis=1, inplace=True)
    data = pd.get_dummies(data)
    
    return data, target
    

----

## metric selection

The tutorial used **F-beta score** where   
 * precision, what proportion of people classified as '>50K' actually makes more than 50K, rather than  
 * recall, what proportion of people that makes more than 50K actually get classified as '>50K'.  

Wth $\beta = 0.5$, which places more emphasis on precision, we are not asking the right question. It's OK to wrongly classify someone as '>50K' -- all you gonna lose is a phone call or a pamphlet; on the other hand, we do want to identify as many people makes more than 50K as possible, for they are the ones likely to donate. Therefore I am changing the $\beta$ value to 2.

In [5]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
def xy_split(data, target, random_state):
    X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    test_size = 0.2, 
                                                    random_state = random_state)
    # Show the results of the split
    print("Training set has {} samples.".format(X_train.shape[0]))
    print("Testing set has {} samples.".format(X_test.shape[0]))

    return X_train, X_test, y_train, y_test

In [32]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

def train_predict(data, target, clf):
    
    X_train, X_test, y_train, y_test = xy_split(data, target, 0)
    
    clf = clf
    clf_name = clf.__class__.__name__
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    print('\nThe confusion matrix looks like: ')
    print(cm)
    #sns.heatmap(cm, annot=True, fmt='.2g')
    
    print('\nThe classification report looks like: ')
    print(classification_report(y_test, y_pred))

    print("Final F-score with beta={} on the testing data: {:.4f}".format(2, fbeta_score(y_test, y_pred, beta=2))) 

In [33]:
tmp, target = ft_engineer(data.copy(), 
                          capital=False, 
                          workclass=False, 
                          occupation=False, 
                          marital=False, 
                          relationship=False, 
                          race=False, 
                          drop_native_country=False)

tmp.shape

C:\Users\Kebei\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(45222, 103)

In [34]:
train_predict(tmp, target, LogisticRegression(solver='liblinear', class_weight='balanced'))

Training set has 36177 samples.
Testing set has 9045 samples.

The confusion matrix looks like: 
[[5385 1455]
 [ 355 1850]]

The classification report looks like: 
              precision    recall  f1-score   support

           0       0.94      0.79      0.86      6840
           1       0.56      0.84      0.67      2205

   micro avg       0.80      0.80      0.80      9045
   macro avg       0.75      0.81      0.76      9045
weighted avg       0.85      0.80      0.81      9045

Final F-score with beta=2 on the testing data: 0.7629


In [31]:
train_predict(tmp, target, LogisticRegression(solver='liblinear'))

Training set has 36177 samples.
Testing set has 9045 samples.

The confusion matrix looks like: 
[[6301  539]
 [ 891 1314]]

The classification report looks like: 
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      6840
           1       0.71      0.60      0.65      2205

   micro avg       0.84      0.84      0.84      9045
   macro avg       0.79      0.76      0.77      9045
weighted avg       0.84      0.84      0.84      9045

Final F-score with beta=2 on the testing data: 0.6832


**Observation**  
 * had `target = 1-target` by mistake and the scores are greatly improved; must be an indication on unblanced classes

###  upsampling

In [ ]:
tmp, target = ft_engineer(data.copy(), 
                          capital=True, 
                          workclass=True, 
                          occupation=True, 
                          marital=True, 
                          relationship=True, 
                          race=True, 
                          drop_native_country=True)

X_train, X_test, y_train, y_test = xy_split(tmp, target, 0)

# before SMOTE
u, ct = np.unique(y_train, return_counts=True)
print('before SMOTE:')
print("number of instances for class {} is {}".format(u[0], ct[0]))
print("number of instances for class {} is {}".format(u[1], ct[1]))

from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)

X_upsamp, y_upsamp = os.fit_sample(X_train, y_train)
X_upsamp = pd.DataFrame(data=X_upsamp,columns=X_train.columns )
#y_upsamp = pd.DataFrame(data=y_upsamp,columns=['y'])

# after SMOTE
u, ct = np.unique(y_upsamp, return_counts=True)

print('after SMOTE:')
print("number of instances for class {} is {}".format(u[0], ct[0]))
print("number of instances for class {} is {}".format(u[1], ct[1]))

# double check that SMOTE works as expected
# The original X_train and y_train was not changed
X_diff = (X_train['age'].values-X_upsamp.iloc[:X_train.shape[0]]['age'].values).sum()
y_diff = (y_train - y_upsamp[:X_train.shape[0]]).sum()
print('The original instance are intact: {}'.format((X_diff==0)&(y_diff==0)))
# The interpolated are on the tail
(y_upsamp[X_train.shape[0]:]==0).sum()

Training set has 36177 samples.
Testing set has 9045 samples.
before SMOTE:
number of instances for class 0 is 27174
number of instances for class 1 is 9003


C:\Users\Kebei\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


after SMOTE:
number of instances for class 0 is 27174
number of instances for class 1 is 27174
The original instance are intact: True


0

In [ ]:
sns.countplot(X_upsamp.iloc[X_train.shape[0]:]['age'])

In [ ]:
sns.countplot(X_train[y_train==1]['age'])

In [ ]:
X_train.tail(10)

In [ ]:
(X_train['age'].values-X_upsamp.iloc[:36177]['age'].values).sum()

In [ ]:
y_upsamp[36168:36300]

In [ ]:
X_train['age'] == X_upsamp.iloc[:36177]['age']

In [ ]:
y_
idx_rdm = np.random.choice(np.where(y_upsamp==0)[0], 10, replace=False)
X_upsamp.iloc[idx_rdm]
#X_train[idx_rdm]

In [ ]:
clf = LogisticRegression()
#clf_name = clf.__class__.__name__

clf.fit(os_data_X, os_data_y)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print('\nThe confusion matrix looks like: ')
print(cm)

print('\nThe classification report looks like: ')
print(classification_report(y_test, y_pred))

print("Final F-score with beta={} on the testing data: {:.4f}".format(2, fbeta_score(y_test, y_pred, beta=2))) 

###  downsampling

In [ ]:
idx_2b_rmvd = np.where(y_train==0)[0]
idx_2b_rmvd = np.random.choice(idx_2b_rmvd, (y_train==0).sum() - (y_train==1).sum(), replace=False)

In [ ]:
X_train_downsamp = X_train.iloc[~idx_2b_rmvd]
y_train_downsamp = y_train[~idx_2b_rmvd]

In [ ]:
train_predict(tmp, target, LogisticRegression())

In [ ]:
clf = LogisticRegression()
#clf_name = clf.__class__.__name__

clf.fit(X_train_downsamp, y_train_downsamp)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print('\nThe confusion matrix looks like: ')
print(cm)

print('\nThe classification report looks like: ')
print(classification_report(y_test, y_pred))

print("Final F-score with beta={} on the testing data: {:.4f}".format(2, fbeta_score(y_test, y_pred, beta=2))) 

### upsampling and downsampling

### cost weighted algorithm